# 背景

在东财app中，选择近一个月机构调研次数3次以上；股吧人气排名前1000名；换手率>5%；10日涨跌幅0%~10%的股票进行跟踪，反复炒作。

## 买入

30分钟线均处于低位，并且30分钟得到一周期确认。注意如果日线处于明显的下降通道、特别是缓降通道时，还要参考均线压制信号。比如东土科技在11月2日14：30点给出RSI低位确认信号，但由于上方多均线压制，日线看RSI并不超跌（30.2, 成为最低点的概率为16%)，所以还要再等，到2021年11月4日尾盘更合适。

<img src="https://images.jieyu.ai/images/202111/20211104215554.png" width="350"/>

完全符合上述pattern的有：金杯电工（11月3日）、运达股份（11月3日）、芯能科技（11月3日）、华明装备（11月3日）、东方电热（11月3日）、合兴股份（10月29日）等。

英科医疗（11月2日）RSI处于低位，次日涨14%。但下一周期上涨幅度小于1%，不知道能否算确认。类似RSI30处于低位后，第二天30分钟类拉涨停的还有一些。


## 卖出

日线处于高位并且30分钟也处于高位

## 淘汰

股票入池后60日移除

In [1]:
from alpha.notebook import *
import traceback
import json

from alpha.core.rsi_stats import RsiStats, rsiday, rsi30
await init_notebook()

events = {
    tf.date2int(arrow.now()): {}
}

In [2]:
def load_fired_events(path="/apps/alpha/data/机构调研-信号.txt"):
    # 当天和头一天已报过的信号不重复报
    today = tf.date2int(arrow.now())

    with open(path, "r") as f:
        events = json.load(f)
        
    today = tf.date2int(arrow.now())
    if today not in events:
        events[today] = {}
        
    return events

def save_fire_events(path="/apps/alpha/data/机构调研-信号.txt"):
    global events
    with open(path, "w") as f:
        json.dump(events, f)
    
def is_fired(name):
    global events
    
    end = arrow.now().date()
    
    start = tf.date2int(tf.day_shift(end, -1))
    end = tf.date2int(end)
    
    if name in events[end]:
        return True
    
    if start in events and name in events[start]:
        return True
        
    return False

def remember(name, params: List):
    end = tf.date2int(arrow.now())
    events[end][name] = params
    
def print_list():
    df = load_pool()
    groups = df.groupby('date').groups
    print("最新监控名单:\n")
    for tm in groups:
        index = groups[tm].tolist()
        print(tm, " ".join(df.loc[index]["name"].tolist()))


In [3]:
def load_pool(path="/apps/alpha/data/机构调研.txt"):
    with open(path, "r") as f:
        content = f.readlines()
        
    stocks = {}
    for line in content:
        line = line.strip()
        if line.startswith("#") or len(line) == 0:
            continue
        
        tm, *names = line.split(" ")
        for name in names:
            if name in stocks:
                continue
                
            stocks[name] = arrow.get(tm).date()
            
    df = pd.DataFrame().from_dict(stocks.items())
    df.columns = ["name", "date"]
    
    time_mark = arrow.now().shift(days=-30).date()
    df = df[df.date > time_mark]
        
    return df

In [4]:
def trigger(code, name, bars, results, frame_type):
    """30分钟RSI低点确认，或者由跌转升
    
    低点确认原则：
    1. RSI低位(prsi < 10%)且有长下影
    2. 前5周期内有RSI低位且自低位后，有1%以上涨幅
    3. RSI不在低位，但系30日以内最低点
    4. 其它可以确认低位的方法
    """
    if is_fired(name): #一个信号两天只报一次
        return
    
    close = bars["close"]
    
    rsi = relative_strength_index(close, 6)
    min_rsi = np.min(rsi[-5:])
    ipos = np.argmin(rsi[-5:])
    
    rf = reversing(close)
        
    # 低位rsi
    pmin_rsi = rsi30.get_proba(code, min_rsi)

    low_rsi = (pmin_rsi is not None and (pmin_rsi < 0.1 or abs(min_rsi - np.min(rsi) < 1e-4)))
    
    # 出现低位rsi后，上涨 > 1%,但不追涨（后面跌下来可以继续报）
    low = bars["low"][-5:]
    adv = round((close[-1] / low[ipos - 5] - 1),3)
    is_up = (0.03 >= adv >= 0.01)

    frame = str(bars["frame"][-1])[5:-9]
    #print(name, frame, pmin_rsi, min_rsi, adv)

    if low_rsi and is_up or rf == 1:
        remember(name, [name, code, frame, pmin_rsi, min_rsi, adv, rf])
        results.append([name, code, frame, pmin_rsi, min_rsi, adv, rf])

In [5]:
async def strategy(tm=None):
    records = load_pool()
    names = records["name"].tolist()
    codes = [name_to_code(name) for name in names]
    #codes = ["002270.XSHE"]
    r2 = await scan(trigger, 80, '30m', tm = tm, codes=codes)
    if len(r2):
        columns = ["name", "code", "frame", "PRSI", "RSI", "adv", "d2"]
        df = pd.DataFrame(r2, columns=columns)
        display(df)
        await nb_say("机构调研股有新的信号。")

# 每日监控

In [6]:
load_fired_events()
print_list()
await scheduler(strategy)
print("结束一天的机构调研股监控。")

最新监控名单:

2021-10-14 科博达 芯能科技 英科医疗 天能股份 润建股份 聚飞光电 拓维信息 新五丰
2021-10-15 泉峰汽车 华中数控 合兴股份 中国黄金 朗特智能 晓鸣股份
2021-10-16 泰晶科技 华明装备 国民技术 中天火箭 瑞丰光电 鑫铂股份 伊戈尔 拓邦股份
2021-10-18 金杯电工 恒星科技 东方电热 汤姆猫 日发精机 天齐锂业 运达股份 湘潭电化 四川双马 味知香
2021-10-19 唐人神 海达股份 顺络电子 易事特 川发龙蟒 硅宝科技 航发控制 聚灿光电
2021-10-20 云海金属 延安必康 苏州固锝 云图控股 宝通科技 鲁北化工
2021-10-21 东土科技 楚天科技 新亚电子 东南网架 国林科技
2021-11-04 中闽能源 佳禾食品 智光电气 申昊科技 联合光电 盛弘股份 通裕重工 达瑞电子 珠海冠宇 九洲集团 西麦食品 汉威科技 新乳业 雷曼光电 博彦科技
=========== 14:45============
结束一天的机构调研股监控。
